# Exomoon Candidate Filtering: Observational and Dynamical Constraints


## 1. Introduction
This notebook implements a two-stage filtering pipeline to identify promising 
exoplanetary systems for exomoon searches. 
The filters combine observational constraints from upcoming missions 
(PLATO and HWO) with dynamical stability criteria for potential moons.


## 2. Data Acquisition
- Load the catalogue of known exoplanets from [exoplanet.eu](http://exoplanet.eu).
- Restrict to confirmed planets with masses below 60 MJup, including 1σ uncertainties.
- Convert all relevant parameters to consistent units (Earth or Jupiter units, AU, etc.).


In [22]:
import pandas as pd

# Read the catalogue
df = pd.read_csv("exoplanets.csv")

# Copies to work on
df_plato = df.copy()
df_hwo = df.copy()

# AUXILIAR FUNCTIONS
def filter_with_unknown(df, col, min_val=None, max_val=None):
    mask = pd.Series(True, index=df.index)
    unknown = pd.Series(False, index=df.index)
    if col in df.columns:
        values = df[col]
        unknown = values.isna()
        if min_val is not None:
            mask &= (values >= min_val) | (values.isna())
        if max_val is not None:
            mask &= (values <= max_val) | (values.isna())
    return mask, unknown

# Show the 1st 5 rows
print(df.head())
print(f"Total rows loaded: {len(df)}")

        name planet_status   mass  mass_error_min  mass_error_max  mass_sini  \
0  109 Psc b     Confirmed  5.743           0.289           1.011     6.3830   
1  112 Psc b     Confirmed    NaN           0.005           0.004     0.0330   
2  112 Psc c     Confirmed  9.866           1.781           3.190        NaN   
3   11 UMi b     Confirmed    NaN           1.100           1.100    11.0873   
4  14 And Ab     Confirmed    NaN           0.230           0.230     4.6840   

   mass_sini_error_min  mass_sini_error_max  radius  radius_error_min  ...  \
0                0.078                0.078   1.152               NaN  ...   
1                0.005                0.004     NaN               NaN  ...   
2                  NaN                  NaN     NaN               NaN  ...   
3                1.100                1.100     NaN               NaN  ...   
4                0.230                0.230     NaN               NaN  ...   

   star_sp_type  star_age  star_age_error_min  sta

## 3. Observational Filtering
Two mission-specific observational filters are applied:
- **PLATO constraints** (transit-based detection)
- **HWO constraints** (direct imaging in reflected light)


### 3.1 PLATO filters

In [23]:
# --- PLATO CRITERIA ---
plato_unknown_flags = {}

# 1. Teff: 4500–6500 K
mask, unk = filter_with_unknown(df_plato, "star_teff", 4500, 6500)
df_plato = df_plato[mask]
plato_unknown_flags["unknown_teff"] = unk

# 2. Radius > 0.5 RJup
mask, unk = filter_with_unknown(df_plato, "radius", 0.5, None)
df_plato = df_plato[mask]
plato_unknown_flags["unknown_radius"] = unk

# 3. Period 10–300 days
mask, unk = filter_with_unknown(df_plato, "orbital_period", 10, 300)
df_plato = df_plato[mask]
plato_unknown_flags["unknown_period"] = unk

# 4. Impact parameter < 0.7
mask, unk = filter_with_unknown(df_plato, "impact_parameter", None, 0.7)
df_plato = df_plato[mask]
plato_unknown_flags["unknown_impact"] = unk

# 5. V magnitude < 10
mask, unk = filter_with_unknown(df_plato, "mag_v", None, 10)
df_plato = df_plato[mask]
plato_unknown_flags["unknown_mag_v"] = unk

# Add columns of unknown
for k, v in plato_unknown_flags.items():
    df_plato[k] = v



In [24]:
print("Unkown data star_radius:", df_hwo['star_radius'].isna().sum())
print("Unkown data star_teff:", df_hwo['star_teff'].isna().sum())
print("Unkown data semi_major_axis:", df_hwo['semi_major_axis'].isna().sum())


Unkown data star_radius: 625
Unkown data star_teff: 504
Unkown data semi_major_axis: 1547


### 3.2 HWO filters

In [25]:
import numpy as np

# Sun temperature constant (in Kelvin)
T_sun = 5772

# Compute the approximate stellar luminosity (in solar units)
df_hwo['star_lum'] = (df_hwo['star_radius']**2) * ((df_hwo['star_teff'] / T_sun)**4)

# Compute lower and opper limits of the HZ (in AU)
df_hwo['hz_inner'] = 0.95 * np.sqrt(df_hwo['star_lum'])
df_hwo['hz_outer'] = 1.67 * np.sqrt(df_hwo['star_lum'])

# --- HWO CRITERIA ---
hwo_unknown_flags = {}

# 1. Distance < 20 pc
mask, unk = filter_with_unknown(df_hwo, "star_distance", None, 20)
df_hwo = df_hwo[mask]
hwo_unknown_flags["unknown_distance"] = unk

# 2. Semi-major axis inside HZ (hz_inner <= sma <= hz_outer)
mask_sma = (df_hwo['semi_major_axis'] >= df_hwo['hz_inner']) & (df_hwo['semi_major_axis'] <= df_hwo['hz_outer'])
unk_sma = df_hwo['semi_major_axis'].isna()
mask = mask_sma | unk_sma  # permitir unknown semi_major_axis si quieres

df_hwo = df_hwo[mask]
hwo_unknown_flags["unknown_sma"] = unk_sma

# 3. Radius 
# Radius between 0.044 and 0.133 RJup (approx. 0.5-1.5 R_earth)
mask, unk = filter_with_unknown(df_hwo, "radius", 0.044, 0.133)
df_hwo = df_hwo[mask]
hwo_unknown_flags["unknown_radius"] = unk

# 4. V magnitude < 10
mask, unk = filter_with_unknown(df_hwo, "mag_v", None, 10)
df_hwo = df_hwo[mask]
hwo_unknown_flags["unknown_mag_v"] = unk

# 5. Teff: 4500–6500 K
mask, unk = filter_with_unknown(df_hwo, "star_teff", 4500, 6500)
df_hwo = df_hwo[mask]
hwo_unknown_flags["unknown_teff"] = unk



In [26]:
print("Candidates before the HZ filter:", len(df_hwo))
mask_sma = (df_hwo['semi_major_axis'] >= df_hwo['hz_inner']) & (df_hwo['semi_major_axis'] <= df_hwo['hz_outer'])
print("Candidates inside HZ:", mask_sma.sum())


Candidates before the HZ filter: 14
Candidates inside HZ: 8


In [33]:
mask_radius, _ = filter_with_unknown(df_hwo, "radius", 0.071, 0.111)
print("Candidates with Earth-radius:", mask_radius.sum())


Candidates with Earth-radius: 14


In [28]:
# Add unknown-columns
for k, v in hwo_unknown_flags.items():
    df_hwo[k] = v

# Save results
df_plato.to_csv("plato_candidates.csv", index=False)
df_hwo.to_csv("hwo_candidates.csv", index=False)

print(f"PLATO candidates saved: {len(df_plato)}")
print(f"HWO candidates saved: {len(df_hwo)}")


PLATO candidates saved: 610
HWO candidates saved: 14


## 4. Dynamical Filtering
Three dynamical stability criteria are implemented:
1. Hill stability (prograde and retrograde orbits),
2. Roche limit (avoidance of tidal disruption),
3. Laplace radius (stellar perturbations).


In [29]:
# Constants
M_sun_kg = 1.98847e30
M_jup_kg = 1.89813e27
M_earth_kg = 5.97219e24
R_jup_m = 69911e3
R_earth_m = 6371e3
AU_m = 1.495978707e11

# Test parameters
moon_masses_mearth = np.array([0.001, 0.01, 0.1, 0.5, 1.0])  # M_earth
moon_eccs = np.array([0.0, 0.1, 0.3, 0.5])

# Coefs for reduced Hill (use prograde by default)
B_pro = 0.49
b_pro = 1.03

def planet_mass_from_radius(Rp_Rj):
    # simple fallback: if we dont have M_p, dont try 
    return np.nan

def check_moon_stability(row,
                         use_prograde=True,
                         moon_masses=moon_masses_mearth,
                         moon_eccs=moon_eccs):
    # Ree DataFrame values (check the column names)
    try:
        M_p = row['mass']            # in M_jup (or NaN)
        R_p = row['radius']          # in R_jup
        a_p = row['semi_major_axis'] # in AU
        e_p = row.get('eccentricity', 0.0) or 0.0
        M_star = row.get('star_mass') # in M_sun
    except Exception:
        return {'possible': False}

    # We need minimum values
    if np.isnan(M_p) or np.isnan(R_p) or np.isnan(a_p) or np.isnan(M_star):
        return {'possible': False}

    # convert to IS
    M_p_kg = M_p * M_jup_kg
    R_p_m = R_p * R_jup_m
    a_p_m = a_p * AU_m
    M_star_kg = M_star * M_sun_kg

    # CLassic and reduced Hill
    hill = a_p_m * (M_p_kg / (3.0 * M_star_kg))**(1/3)
    if use_prograde:
        B, b = B_pro, b_pro
    else:
        B, b = 0.93, 1.08
    hill_reduced = B * a_p_m * (1 - b * e_p) * (M_p_kg / (3.0 * M_star_kg))**(1/3)

    # Laplace limit (in m)
    laplace = 0.5 * a_p_m * (M_star_kg / M_p_kg)**(1/3)

    # Save viable intervals
    viable_cases = []

    for Mm_earth in moon_masses:
        M_m_kg = Mm_earth * M_earth_kg
        # Roche (en m)
        r_roche = 2.2 * R_p_m * (M_p_kg / M_m_kg)**(1/3)

        for e_m in moon_eccs:
            # conditions in a_m (in m)
            # a_min by Roche: a_m (1 - e_m) > r_roche  => a_m > r_roche / (1 - e_m)
            if (1 - e_m) <= 0:
                continue
            a_min = r_roche / (1 - e_m)

            # a_max by reduced Hill: a_m (1 + e_m) < hill_reduced => a_m < hill_reduced / (1 + e_m)
            a_max_hill = hill_reduced / (1 + e_m)

            # respect Laplace: a_m < laplace
            a_max = min(a_max_hill, laplace)

            # Physical test: a_min < a_max and both less than r_Hill classical etc.
            if a_min < a_max and a_max > 0:
                # convert a_min/a_max to AU for saving
                viable_cases.append({
                    'Mm_earth': Mm_earth,
                    'e_m': e_m,
                    'a_min_AU': a_min / AU_m,
                    'a_max_AU': a_max / AU_m,
                    'r_roche_AU': r_roche / AU_m,
                    'hill_reduced_AU': hill_reduced / AU_m,
                    'laplace_AU': laplace / AU_m
                })

    # If there is at least one viable case, mark it as true
    possible = len(viable_cases) > 0
    # Give back a resume
    if possible:
        # Ordering by bigger semi-major axis
        widths = [c['a_max_AU'] - c['a_min_AU'] for c in viable_cases]
        best = viable_cases[np.argmax(widths)]
    else:
        best = None

    return {'possible': possible, 'n_viable': len(viable_cases), 'best_case': best, 'viable_cases': viable_cases}




## 5. Results
- Number of PLATO candidates after filtering,
- Number of HWO candidates after filtering,
- List of stable systems satisfying all criteria.


In [30]:
# Assumed moon mass (M_jup).
M_m_assumed_list = [0.0005, 0.001, 0.005, 0.01, 0.02, 0.05]  # en M_jup (~0.16 M_earth ... 15 M_earth)
# Assumed moon excentricities
moon_eccs = [0.0, 0.1, 0.3]
# If no planet excentricity value, use this one
e_p_default = 0.0

# Coeficients for reduced Hill
B_pro, b_pro = 0.49, 1.03
B_ret, b_ret = 0.93, 1.08

# Column names
# We assume: planet mass en M_jup -> 'mass' o 'pl_mass' ; radius en R_jup -> 'radius' ; 
# semi-major axis en AU -> 'semi_major_axis' ; eccentricity -> 'eccentricity' ; star mass en M_sun -> 'star_mass'
POSSIBLE_PLANET_MASS_COLS = ['mass', 'pl_mass', 'planet_mass', 'M_p', 'mpl']
POSSIBLE_PLANET_RADIUS_COLS = ['radius', 'pl_radius', 'R_p', 'radius_jup']
POSSIBLE_SMA_COLS = ['semi_major_axis', 'a', 'a_p', 'semi_major']
POSSIBLE_ECC_COLS = ['eccentricity', 'e', 'e_p']
POSSIBLE_STAR_MASS_COLS = ['star_mass', 'st_mass', 'M_*', 'mass_star', 'Mstar']


# Additional functions
def pick_column(df, possibilities):
    for c in possibilities:
        if c in df.columns:
            return c
    return None

def analyze_df(df, name, save_csv=True):
    # Detect columns
    col_mp = pick_column(df, POSSIBLE_PLANET_MASS_COLS)
    col_rp = pick_column(df, POSSIBLE_PLANET_RADIUS_COLS)
    col_ap = pick_column(df, POSSIBLE_SMA_COLS)
    col_ep = pick_column(df, POSSIBLE_ECC_COLS)
    col_mstar = pick_column(df, POSSIBLE_STAR_MASS_COLS)

    if any([col is None for col in (col_mp, col_rp, col_ap, col_mstar)]):
        print(f"ERROR: faltan columnas importantes para {name}. Detectadas:",
              dict(mp=col_mp, rp=col_rp, ap=col_ap, ep=col_ep, mstar=col_mstar))
        return None

    # Create columns to work on
    df_work = df.copy()
    # normalize temporal names
    df_work = df_work.rename(columns={col_mp: 'M_p', col_rp: 'R_p', col_ap: 'a_p', col_mstar: 'M_star'})
    if col_ep:
        df_work = df_work.rename(columns={col_ep: 'e_p'})

    # flags and numerical columns
    df_work['e_p'] = df_work.get('e_p', np.nan).fillna(e_p_default).astype(float)
    df_work['M_p'] = pd.to_numeric(df_work['M_p'], errors='coerce')
    df_work['R_p'] = pd.to_numeric(df_work['R_p'], errors='coerce')
    df_work['a_p'] = pd.to_numeric(df_work['a_p'], errors='coerce')
    df_work['M_star'] = pd.to_numeric(df_work['M_star'], errors='coerce')

    # Initialize output columns 
    df_out = df_work.copy()
    df_out['rH_pro'] = np.nan
    df_out['rH_ret'] = np.nan
    df_out['rR_Mm_best'] = np.nan   # Roche with the lunar mass that best suits (see below)
    df_out['laplace'] = np.nan

    # Falgs for each criteria (if it exists at least one combination Mm/e_m that works)
    df_out['pass_hill_pro'] = False
    df_out['pass_hill_ret'] = False
    df_out['pass_roche'] = False
    df_out['pass_laplace'] = False
    df_out['pass_all'] = False

    # For each planet, we try the interval of lunar masses and excentricities
    for idx, row in df_out.iterrows():
        M_p = row['M_p']      # asumed in M_jup
        R_p = row['R_p']      # asumed in R_jup
        a_p = row['a_p']      # AU
        M_star = row['M_star']# asumed in M_sun
        e_p = row['e_p']

        # If something essential is missing, skip it.
        if np.isnan(M_p) or np.isnan(R_p) or np.isnan(a_p) or np.isnan(M_star):
            continue

        # For consistency in units: M_star in M_sun, M_p in M_jup; convert M_star to M_jup for Hill's formula:
        M_star_Mjup = M_star * 1047.348644  # 1 M_sun ≈ 1047.348644 M_jup

        # CLassic and reduced Hill  (in AU)
        rH_classic = a_p * (M_p / (3.0 * M_star_Mjup))**(1/3)
        rH_pro = B_pro * a_p * (1 - b_pro * e_p) * (M_p / (3.0 * M_star_Mjup))**(1/3)
        rH_ret = B_ret * a_p * (1 - b_ret * e_p) * (M_p / (3.0 * M_star_Mjup))**(1/3)
        laplace_lim = 0.5 * a_p * (M_star_Mjup / M_p)**(1/3)

        # We save the values
        df_out.at[idx, 'rH_pro'] = rH_pro
        df_out.at[idx, 'rH_ret'] = rH_ret
        df_out.at[idx, 'laplace'] = laplace_lim

        # Evaluate for all moon masses and excentricities
        any_roche_ok = False
        any_hill_pro_ok = False
        any_hill_ret_ok = False
        any_laplace_ok = False
        any_all_ok = False
        best_roche_ratio = np.inf
        best_case = None

        for M_m_assumed in M_m_assumed_list:
            # Roche limit: rR = 2.2 R_p (M_p / M_m)^(1/3)
            # Atention: R_p in R_jup, we want AU units for comparison => convert R_jup -> AU
            R_jup_AU = 69911e3 / 1.495978707e11  # ≈ 0.000467 AU
            rR = 2.2 * R_p * R_jup_AU * (M_p / M_m_assumed)**(1/3)

            # Try moon excentricities
            for e_m in moon_eccs:
                if (1 - e_m) <= 0:
                    continue
                a_min = rR / (1 - e_m)                # minimal semi-major axis (AU) for the pericenter > Roche
                a_max_hill_pro = rH_pro / (1 + e_m)
                a_max_hill_ret = rH_ret / (1 + e_m)
                a_max = min(a_max_hill_pro, rH_classic, laplace_lim)

                # Individual conditions
                roche_ok = a_min <  a_max  # exists semi-major axis that does not violate Roche and Hill/Laplace
                hill_pro_ok = a_min < a_max_hill_pro
                hill_ret_ok = a_min < a_max_hill_ret
                lap_ok = a_min < laplace_lim

                any_roche_ok = any_roche_ok or roche_ok
                any_hill_pro_ok = any_hill_pro_ok or hill_pro_ok
                any_hill_ret_ok = any_hill_ret_ok or hill_ret_ok
                any_laplace_ok = any_laplace_ok or lap_ok
                if roche_ok and hill_pro_ok and hill_ret_ok and lap_ok:
                    any_all_ok = True

                # For diagnosis: ratio rR / rH_pro (when smaller, better)
                ratio = (rR / rH_pro) if rH_pro>0 else np.inf
                if ratio < best_roche_ratio:
                    best_roche_ratio = ratio
                    best_case = dict(Mm=M_m_assumed, e_m=e_m, rR=rR, a_min=a_min,
                                     a_max_hill_pro=a_max_hill_pro, a_max_hill_ret=a_max_hill_ret,
                                     laplace=laplace_lim, ratio=ratio)

        # Save flags
        df_out.at[idx, 'pass_roche'] = any_roche_ok
        df_out.at[idx, 'pass_hill_pro'] = any_hill_pro_ok
        df_out.at[idx, 'pass_hill_ret'] = any_hill_ret_ok
        df_out.at[idx, 'pass_laplace'] = any_laplace_ok
        df_out.at[idx, 'pass_all'] = any_all_ok

        # Save best diagnosis case 
        if best_case is not None:
            df_out.at[idx, 'best_Mm_Mjup'] = best_case['Mm']
            df_out.at[idx, 'best_e_m'] = best_case['e_m']
            df_out.at[idx, 'best_rR_AU'] = best_case['rR']
            df_out.at[idx, 'best_ratio_rR_rH_pro'] = best_case['ratio']

    # Numerical summary
    summary = {
        'n_total': len(df_out),
        'n_pass_roche': int(df_out['pass_roche'].sum()),
        'n_pass_hill_pro': int(df_out['pass_hill_pro'].sum()),
        'n_pass_hill_ret': int(df_out['pass_hill_ret'].sum()),
        'n_pass_laplace': int(df_out['pass_laplace'].sum()),
        'n_pass_all': int(df_out['pass_all'].sum())
    }

    print(f"\n Summary {name}:")
    for k,v in summary.items():
        print(f"  {k}: {v}")

    if save_csv:
        outname = f"{name.lower().replace(' ','_')}_stability_breakdown.csv"
        df_out.to_csv(outname, index=False)
        print(f"  -> saved CSV: {outname}")

        #  Filter only candidates that fullfill all the criteria
        surviving = df_out[df_out['pass_all'] == True]

        if surviving.empty:
            print("\n⚠ There are no candidates that fullfill all the criteria")
        else:
            cols_preview = [
                'M_p', 'R_p', 'a_p', 'M_star', 'e_p',
                'rH_pro', 'rH_ret', 'laplace', 
                'best_Mm_Mjup', 'best_e_m', 'best_rR_AU',
                'pass_roche', 'pass_hill_pro', 'pass_hill_ret', 'pass_laplace', 'pass_all'
            ]
            cols_preview = [c for c in cols_preview if c in surviving.columns]

            print("\n Candidates that fullfill all the criteria:")
            print(surviving[cols_preview].to_string(index=False))

            # Save CSV only with stable candidates
            stable_outname = f"{name.lower().replace(' ','_')}_stable_candidates.csv"
            surviving.to_csv(stable_outname, index=False)
            print(f"  -> CSV with saved stable candidates: {stable_outname}")

    return df_out, summary


# Run for both CSV

if __name__ == "__main__":
    df_hwo = pd.read_csv("hwo_candidates.csv")
    df_plato = pd.read_csv("plato_candidates.csv")

    hwo_out, hwo_sum = analyze_df(df_hwo, "HWO", save_csv=True)
    plato_out, plato_sum = analyze_df(df_plato, "PLATO", save_csv=True)




 Summary HWO:
  n_total: 14
  n_pass_roche: 0
  n_pass_hill_pro: 0
  n_pass_hill_ret: 0
  n_pass_laplace: 0
  n_pass_all: 0
  -> saved CSV: hwo_stability_breakdown.csv

⚠ There are no candidates that fullfill all the criteria

 Summary PLATO:
  n_total: 610
  n_pass_roche: 27
  n_pass_hill_pro: 27
  n_pass_hill_ret: 32
  n_pass_laplace: 36
  n_pass_all: 27
  -> saved CSV: plato_stability_breakdown.csv

 Candidates that fullfill all the criteria:
     M_p     R_p        a_p  M_star    e_p     rH_pro     rH_ret     laplace  best_Mm_Mjup  best_e_m  best_rR_AU  pass_roche  pass_hill_pro  pass_hill_ret  pass_laplace  pass_all
 4.00000 0.96000   43.50000   1.220 0.0000   2.161961   4.103314  148.681283          0.05       0.0    0.004253        True           True           True          True      True
 0.03043 0.52990    0.14800   1.150 0.0740   0.001363   0.002577    2.521898          0.05       0.0    0.000462        True           True           True          True      True
11.00000 1.3

## 6. Cross-check with PLATO's field of view
In addition to applying dynamical and tidal stability criteria, we verified whether the remaining candidates are observable by the PLATO mission. 
For this purpose, we adopted the official LOPS2 field defined by ESA (centered at RA = 95.31°, Dec = –47.887°, with an extent of 49° × 49°). 
We implemented a simple geometric filter that checks if the sky coordinates of each candidate fall within this rectangular field of view. 
The output provides a list of candidates that could, in principle, be monitored by PLATO.


In [31]:
# Official PLATO LOPS2 field (according to ESA, 49°x49°)
PLATO_FIELD = {
    "ra_center": 95.31,     # in degrees
    "dec_center": -47.887,  # in degrees
    "width": 49.0,          # in degrees
    "height": 49.0          # in degrees
}

def in_rectangle(ra, dec, field):
    """Check if an object is inside the rectangular FOV of PLATO (LOPS2)."""
    if pd.isna(ra) or pd.isna(dec):
        return False
    
    ra_min = field['ra_center'] - field['width']/2
    ra_max = field['ra_center'] + field['width']/2
    dec_min = field['dec_center'] - field['height']/2
    dec_max = field['dec_center'] + field['height']/2
    
    return (ra_min <= ra <= ra_max) and (dec_min <= dec <= dec_max)

def check_plato_visibility(stable_csv="plato_stable_candidates.csv"):
    # Read stable candidates
    df = pd.read_csv(stable_csv)

    # Verify required columns
    if not {"ra","dec"}.issubset(df.columns):
        raise ValueError("The CSV must contain 'ra' and 'dec' columns in degrees")

    # Mark visibility
    df['observable_by_PLATO'] = df.apply(lambda r: in_rectangle(r['ra'], r['dec'], PLATO_FIELD), axis=1)

    # Filter visible candidates
    visible_df = df[df['observable_by_PLATO']].copy()

    # Save results
    visible_df.to_csv("plato_stable_candidates_visible.csv", index=False)
    print(f"✓ {len(visible_df)} visible candidates saved to plato_stable_candidates_visible.csv")
    print(visible_df[['name','ra','dec','observable_by_PLATO']])

    return visible_df

if __name__ == "__main__":
    check_plato_visibility()


✓ 2 visible candidates saved to plato_stable_candidates_visible.csv
         name       ra        dec  observable_by_PLATO
17  NGTS-29 b  70.9975 -39.906667                 True
23  TOI-201 b  87.4000 -54.910833                 True


## 7. Conclusions
This pipeline demonstrates the importance of combining observational constraints 
with dynamical stability to identify realistic targets for exomoon searches. 
Future improvements could include:
- updated catalogues,
- more detailed planet/moon interior models,
- extended eccentricity and inclination ranges.


## 8. Repository Information
- Author: Alicia Pérez Rodrigo
- Repository: https://github.com/Aliper02/Exomoons_Catalogue
